In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re

# 크롬 브라우저 열기
driver = webdriver.Chrome('C:/Temp/chromedriver')

driver.get("https://www.instagram.com/accounts/login/")
time.sleep(2)

email = '본인 아이디 '
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = '비밀번호' 
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

In [30]:
def insta_searching(word):
    url = "https://www.instagram.com/explore/tags/" + word
    return url
import time

word = "산수갑산"
url = insta_searching(word)
driver.get(url)

In [29]:
def select_first(driver):
    first = driver.find_element_by_css_selector("div._9AhH0")
    first.click()
    time.sleep(3)
    
select_first(driver)

In [27]:
import re
from bs4 import BeautifulSoup

def get_content(driver):
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    # 본문 내용 
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ' '
    # 해시태그 
    tags = re.findall(r'#[^\s#,\\]+', content)  
    # 작성일자 
    date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    # 좋아요 
    try:
        like = soup.select('div.Nm9Fw > button')[0].text[4:-1]   
    except:
        like = 0
    # 위치
    try: 
        place = soup.select('div.M30cS')[0].text
    except:
        place = ''
   
    data = [content, date, like, place, tags]
    return data

get_content(driver)

['백종원 선생님께서 조보아씨도 안부르고 혼자 먹을 떡볶이다. 생각만 해도 입에 침이 가득 고이게 할 달콤매콤의 정석이다. -📍서울 마포 #마포원조떡볶이 📍-떡볶이 먹게 500원만 달라고 하던 우리시대와 다르게 ‘엄마 떡볶이먹게 만오천원만’ 이라고 말하는 2020년대. 요즘시대에도 2천원짜리 떡볶이가 있다. 심지어 존맛이다.-맵진하고 들큰한 맛의 떡볶이의 정수이다. 고추장과 물엿을 배합한 맛이다. 질좋은 고춧가루를 쓴 것 같다. 청정원 순창고추장 빙의해 맛있게 맵다. 혓바닥이 텔레토비동산이 된듯 하다. 입안에 둥근해가 뜬 맛이다.-어찌보면 취향이 강하게 타는 맛이다. 밀떡을 선호하는 사람도 있고, 시원한 맛의 떡볶이를 선호하는 사람도 있기 때문에. 그치만 많은 사람들에게 긴 시간 사랑을 받아왔기도 하고, 내스타일이다. 아차산 순금이네 떡볶이와 비슷한 재질이다. -10여년만에 방문한 듯 한데 그시절 그맛 그대로의 느낌이다. 먹선생 기억속의 옛날떡볶이의 맛이다. 30년이 넘은 떡볶이 노포 마포원조떡볶이. 코로나시국에도 저렴한 가격으로 끝까지 버텨주셔서 감사합니다.-총평🌕🌕🌕🌗🌑떡볶이는 진짜 맛있습니다. 이모들도 친절해요. 하지만 튀김은…튀김은 안드시는것 추천드립니다. 겉은 따듯하지만 속은 차가운 튀김은…저는 싫어요-어쩌다보니 먹선생도 어느덧 떡볶이에 진심이 되었습니다. 여러분들의 최애 떡볶이집을 댓글로 알려주세요!#먹선생떡볶이존 #먹선생서울존 #먹선생마포존 -#마포맛집 #마포떡볶이 #마포역맛집 #마포구맛집 #떡볶이 #떡볶이맛집 #떡볶이그램 #떡볶이스타그램 #공덕맛집 #백종원맛집 #삼대천왕',
 '2021-10-01',
 0,
 '마포원조떡볶이',
 ['#마포원조떡볶이',
  '#먹선생떡볶이존',
  '#먹선생서울존',
  '#먹선생마포존',
  '#마포맛집',
  '#마포떡볶이',
  '#마포역맛집',
  '#마포구맛집',
  '#떡볶이',
  '#떡볶이맛집',
  '#떡볶이그램',
  '#떡볶이스타그램',
  '#공덕맛집',
  '#백종원맛집',
  '#삼대천왕

In [8]:
def move_next(driver):

    right = driver.find_element_by_css_selector ('a.coreSpriteRightPaginationArrow')
    right.click()
    time.sleep(3)

move_next(driver)

In [31]:
word = "경북여행" 
url = insta_searching(word)

driver.get(url)
time.sleep(3)

select_first(driver)

results = [ ]

target = 500  
for i in range(target):
    
    try:
        data = get_content(driver)  
        results.append(data)
        move_next(driver)
    except:
        time.sleep(2)
        move_next(driver)
    

print(results[:2])

WebDriverException: Message: chrome not reachable
  (Session info: chrome=94.0.4606.71)


In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.columns = ['content','data','like','place','tags']
results_df.to_excel('C:/Users/POASISTOUR/Desktop/경북여행.xlsx')